In [396]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots

# import warnings
# warnings.filterwarnings('ignore')

***All Common***

In [397]:
def find_values_diff(col1, col2):    
    col1_col2 = np.setdiff1d(col1, col2)
    col2_col1 = np.setdiff1d(col2, col1)
    
    return np.concatenate((col1_col2, col2_col1))

#### Начнем исследовать данные

Посмотрю на каждую таблицу и кратенько отмечую интересные моменты

In [398]:
assessments = pd.read_csv('assessments.csv') # Информация об оценках в модулях
resultAssessments = pd.read_csv('studentAssessment.csv') # Сами оценки
courses = pd.read_csv('courses.csv') # Список модулей
studentInfo = pd.read_csv('studentInfo.csv') # Общее инфо о студенте
registration = pd.read_csv('studentRegistration.csv') # Инфа о регистрации студента на модули
studentVle = pd.read_csv('studentVle.csv') # Взаимодействие студента с виртуальной средой обучения
vle = pd.read_csv('vle.csv') # Доступные материалы в виртуальной среде обучения

train_df = pd.read_csv("Train_Who.csv")
test_df = pd.read_csv("Test_Who.csv")

##### Сначала глянем assessments и resultAssessments

In [399]:
# assessments
print(assessments.info())
assessments.sample(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206 entries, 0 to 205
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        206 non-null    object 
 1   code_presentation  206 non-null    object 
 2   id_assessment      206 non-null    int64  
 3   assessment_type    206 non-null    object 
 4   date               195 non-null    float64
 5   weight             206 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 9.8+ KB
None


,code_module,code_presentation,id_assessment,assessment_type,date,weight
148,FFF,2013J,34877,TMA,173.0,25.0
147,FFF,2013J,34876,TMA,131.0,25.0
105,DDD,2014J,25365,TMA,111.0,25.0
41,BBB,2014B,15008,TMA,12.0,5.0
139,FFF,2013J,34880,CMA,236.0,0.0
170,FFF,2014J,34899,TMA,24.0,12.5
3,AAA,2013J,1755,TMA,166.0,20.0


In [400]:
# resultAssessments
print(resultAssessments.info()) # Без пропусков - это хорошо
resultAssessments.sample(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   id_assessment   173912 non-null  int64
 1   id_student      173912 non-null  int64
 2   date_submitted  173912 non-null  int64
 3   is_banked       173912 non-null  int64
dtypes: int64(4)
memory usage: 5.3 MB
None


,id_assessment,id_student,date_submitted,is_banked
64219,24299,614804,244,0
104818,34860,534750,15,0
68122,25337,515945,116,0
17173,14997,588883,44,0
164884,37425,623029,48,0
42766,15022,642405,110,0
101910,30720,654198,62,0


In [401]:
absent_ids = find_values_diff(resultAssessments["id_assessment"], assessments["id_assessment"])
assesments_absent = assessments[assessments["id_assessment"].isin(absent_ids)]
assesments_absent # Остутствуют экзамены

,code_module,code_presentation,id_assessment,assessment_type,date,weight
5,AAA,2013J,1757,Exam,NaN,100.0
11,AAA,2014J,1763,Exam,NaN,100.0
23,BBB,2013B,14990,Exam,NaN,100.0
35,BBB,2013J,15002,Exam,NaN,100.0
47,BBB,2014B,15014,Exam,NaN,100.0
53,BBB,2014J,15025,Exam,NaN,100.0
63,CCC,2014B,40087,Exam,NaN,100.0
73,CCC,2014J,40088,Exam,NaN,100.0
113,EEE,2013J,30713,Exam,235.0,100.0
118,EEE,2014B,30718,Exam,228.0,100.0


In [402]:
group = assessments.groupby("assessment_type").count()
group[group.index == "Exam"] # Приэтом для каких-то 6 экзаменов оценки есть. Ладно(

,code_module,code_presentation,id_assessment,date,weight
assessment_type,,,,,
Exam,24,24,24,13,24


In [403]:
# Объединим эти таблицы
assesmentsFullInfo = pd.merge(
    resultAssessments, 
    assessments,
    how="inner",
    on="id_assessment",
    )
assert(
    (len(assessments.columns) + len(resultAssessments.columns) - 1) == len(assesmentsFullInfo.columns)
    )
assert(
    assesmentsFullInfo.shape[0] == resultAssessments.shape[0]
)
print(assesmentsFullInfo.info())
assesmentsFullInfo.sample(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id_assessment      173912 non-null  int64  
 1   id_student         173912 non-null  int64  
 2   date_submitted     173912 non-null  int64  
 3   is_banked          173912 non-null  int64  
 4   code_module        173912 non-null  object 
 5   code_presentation  173912 non-null  object 
 6   assessment_type    173912 non-null  object 
 7   date               171047 non-null  float64
 8   weight             173912 non-null  float64
dtypes: float64(2), int64(4), object(3)
memory usage: 11.9+ MB
None


,id_assessment,id_student,date_submitted,is_banked,code_module,code_presentation,assessment_type,date,weight
168556,37431,635233,223,0,GGG,2014B,CMA,222.0,0.0
8656,14989,290550,186,0,BBB,2013B,TMA,187.0,18.0
165293,37425,518661,60,0,GGG,2014B,TMA,61.0,0.0
31012,15009,2440756,38,0,BBB,2014B,TMA,40.0,18.0
48529,24285,426616,200,0,CCC,2014B,TMA,200.0,22.0
21440,15001,574320,206,0,BBB,2013J,TMA,208.0,18.0
78015,25349,600817,50,0,DDD,2013J,TMA,53.0,12.5


In [404]:
print(assesmentsFullInfo["is_banked"].value_counts()) # Выкинем этот признак, у него слишком плохое распределение
assesmentsFullInfoV1 = assesmentsFullInfo.drop("is_banked", axis=1) # Будем версионировать DF чтоб была полная воспроизводимость

is_banked
0    172003
1      1909
Name: count, dtype: int64


### Объединим с courses

In [405]:
find_values_diff(
   col1=courses["code_module"] + courses["code_presentation"],
   col2=assesmentsFullInfoV1["code_module"] + assesmentsFullInfoV1["code_presentation"]
) # Ок, можем спокойной мержить

assesmentsFullCourses = pd.merge(
    assesmentsFullInfoV1,
    courses,
    how="inner",
    on=["code_module", "code_presentation"]
)
assert(
    len(assesmentsFullInfoV1.columns) + 1 == len(assesmentsFullCourses.columns)
    )
assert(
    assesmentsFullCourses.shape[0] == assesmentsFullInfoV1.shape[0]
)
assesmentsFullCourses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173912 entries, 0 to 173911
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id_assessment               173912 non-null  int64  
 1   id_student                  173912 non-null  int64  
 2   date_submitted              173912 non-null  int64  
 3   code_module                 173912 non-null  object 
 4   code_presentation           173912 non-null  object 
 5   assessment_type             173912 non-null  object 
 6   date                        171047 non-null  float64
 7   weight                      173912 non-null  float64
 8   module_presentation_length  173912 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 11.9+ MB


#### Проделаем похожее с таблицами о студентах

In [406]:
registration.info()
registration.sample(7) # Регистраций сильно меньше чем оценок, это ок тк по каждому модулю может быть несколько оценок

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code_module        32593 non-null  object 
 1   code_presentation  32593 non-null  object 
 2   id_student         32593 non-null  int64  
 3   date_registration  32548 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1018.7+ KB


,code_module,code_presentation,id_student,date_registration
8282,BBB,2014J,693098,-25.0
19224,DDD,2014J,2008795,-136.0
3894,BBB,2013J,588642,-59.0
11262,CCC,2014J,547128,-155.0
15971,DDD,2013J,604559,-53.0
31080,GGG,2014B,384035,-71.0
22942,FFF,2013B,514969,-96.0


In [407]:
reg_diff = find_values_diff(
   col1=assesmentsFullCourses["code_module"] + assesmentsFullCourses["code_presentation"] + assesmentsFullCourses["id_student"].astype(str),
   col2=registration["code_module"] + registration["code_presentation"] + registration["id_student"].astype(str)
)
len(reg_diff) # Думаю это ок, студенты могли не регистрироваться на курсы вообще
# будем мержить inner, тк если не регистрировался то и не сдавал выходит

6750

In [408]:
assesmentsFullCoursesReg = pd.merge(
    registration,
    assesmentsFullCourses,
    how="outer",
    on=["code_module", "code_presentation", "id_student"]
)
assesmentsFullCoursesReg.info()
assesmentsFullCoursesReg.sample(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180662 entries, 0 to 180661
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   code_module                 180662 non-null  object 
 1   code_presentation           180662 non-null  object 
 2   id_student                  180662 non-null  int64  
 3   date_registration           180614 non-null  float64
 4   id_assessment               173912 non-null  float64
 5   date_submitted              173912 non-null  float64
 6   assessment_type             173912 non-null  object 
 7   date                        171047 non-null  float64
 8   weight                      173912 non-null  float64
 9   module_presentation_length  173912 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 13.8+ MB


,code_module,code_presentation,id_student,date_registration,id_assessment,date_submitted,assessment_type,date,weight,module_presentation_length
38443,BBB,2014B,633212,-29.0,15012.0,150.0,TMA,152.0,18.0,234.0
124377,FFF,2013J,467243,-63.0,34878.0,103.0,CMA,236.0,0.0,268.0
145851,FFF,2014B,630452,-23.0,34892.0,63.0,CMA,227.0,0.0,241.0
11930,BBB,2013B,554083,-24.0,14988.0,159.0,TMA,159.0,18.0,240.0
176777,GGG,2014J,571583,-63.0,37441.0,200.0,CMA,229.0,0.0,269.0
17085,BBB,2013J,408106,-47.0,NaN,NaN,NaN,NaN,NaN,NaN
50182,CCC,2014B,502868,-225.0,24290.0,230.0,Exam,NaN,100.0,241.0


##### Объединим с инфой о студенте

In [409]:
studentInfo.info()
studentInfo.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32593 entries, 0 to 32592
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   code_module           32593 non-null  object
 1   code_presentation     32593 non-null  object
 2   id_student            32593 non-null  int64 
 3   gender                32593 non-null  object
 4   region                32593 non-null  object
 5   highest_education     32593 non-null  object
 6   imd_band              31482 non-null  object
 7   age_band              32593 non-null  object
 8   num_of_prev_attempts  32593 non-null  int64 
 9   studied_credits       32593 non-null  int64 
 10  disability            32593 non-null  object
dtypes: int64(3), object(8)
memory usage: 2.7+ MB


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability
12314,CCC,2014J,650370,F,Scotland,HE Qualification,10-20,0-35,0,90,N
24652,FFF,2013J,554586,M,London Region,A Level or Equivalent,70-80%,0-35,0,120,N
27881,FFF,2014J,331789,M,South East Region,Lower Than A Level,60-70%,0-35,0,60,N
12532,CCC,2014J,679663,M,Scotland,A Level or Equivalent,80-90%,0-35,0,30,N
27746,FFF,2014J,129907,M,East Anglian Region,HE Qualification,90-100%,35-55,0,60,N
19748,EEE,2013J,561787,M,South West Region,Lower Than A Level,60-70%,0-35,0,120,N
12790,CCC,2014J,695732,F,North Region,Post Graduate Qualification,NaN,35-55,0,60,N
25061,FFF,2013J,579257,M,South East Region,A Level or Equivalent,90-100%,0-35,0,60,N
23985,FFF,2013J,133185,M,East Anglian Region,A Level or Equivalent,50-60%,0-35,0,120,N
7794,BBB,2014J,676420,F,South West Region,Lower Than A Level,60-70%,0-35,0,90,N


In [410]:
students_diff = find_values_diff(
   col1=assesmentsFullCoursesReg["code_module"] + assesmentsFullCoursesReg["code_presentation"] + assesmentsFullCoursesReg["id_student"].astype(str),
   col2=studentInfo["code_module"] + studentInfo["code_presentation"] + studentInfo["id_student"].astype(str)
)
len(students_diff) # Такая же картина как с регистрациями - это логично

0

In [411]:
assesmentsCoursesRegStudentInfo = pd.merge(
    assesmentsFullCoursesReg,
    studentInfo,
    how="right",
    on=["code_module", "code_presentation", "id_student"]
)
assesmentsCoursesRegStudentInfo.info() # Shape ok
assesmentsCoursesRegStudentInfo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180662 entries, 0 to 180661
Data columns (total 18 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   code_module                 180662 non-null  object 
 1   code_presentation           180662 non-null  object 
 2   id_student                  180662 non-null  int64  
 3   date_registration           180614 non-null  float64
 4   id_assessment               173912 non-null  float64
 5   date_submitted              173912 non-null  float64
 6   assessment_type             173912 non-null  object 
 7   date                        171047 non-null  float64
 8   weight                      173912 non-null  float64
 9   module_presentation_length  173912 non-null  float64
 10  gender                      180662 non-null  object 
 11  region                      180662 non-null  object 
 12  highest_education           180662 non-null  object 
 13  imd_band      

,code_module,code_presentation,id_student,date_registration,id_assessment,date_submitted,assessment_type,date,weight,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability
0,AAA,2013J,11391,-159.0,1752.0,18.0,TMA,19.0,10.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N
1,AAA,2013J,11391,-159.0,1753.0,53.0,TMA,54.0,20.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N
2,AAA,2013J,11391,-159.0,1754.0,115.0,TMA,117.0,20.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N
3,AAA,2013J,11391,-159.0,1755.0,164.0,TMA,166.0,20.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N
4,AAA,2013J,11391,-159.0,1756.0,212.0,TMA,215.0,30.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180657,GGG,2014J,2684003,-28.0,37437.0,169.0,TMA,173.0,0.0,269.0,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N
180658,GGG,2014J,2684003,-28.0,37438.0,73.0,CMA,229.0,0.0,269.0,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N
180659,GGG,2014J,2684003,-28.0,37439.0,150.0,CMA,229.0,0.0,269.0,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N
180660,GGG,2014J,2684003,-28.0,37440.0,172.0,CMA,229.0,0.0,269.0,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N


### Инфо о взаимодействии с материалами попробуем сгруппировать

In [412]:
vle[["week_from", "week_to"]].count() # Эти признаки рассматривать не будем их меньше 20%
vleV1 = vle.drop(["week_from", "week_to"], axis=1)
vleV1.sample(10)

,id_site,code_module,code_presentation,activity_type
5040,716257,FFF,2013J,oucontent
2257,547547,DDD,2013B,oucontent
1226,768527,BBB,2014B,resource
2154,547751,DDD,2013B,resource
5307,779456,FFF,2014B,page
3053,773396,DDD,2014B,resource
4340,527117,FFF,2013B,questionnaire
2212,547806,DDD,2013B,url
5101,779278,FFF,2014B,questionnaire
3722,551063,EEE,2013J,oucontent


In [413]:
vleV1["activity_type"].value_counts() # Сомнительная конечно таблица, но ок оставим

activity_type
resource          2660
subpage           1055
oucontent          996
url                886
forumng            194
quiz               127
page               102
oucollaborate       82
questionnaire       61
ouwiki              49
dataplus            28
externalquiz        26
homepage            22
ouelluminate        21
glossary            21
dualpane            20
repeatactivity       5
htmlactivity         4
sharedsubpage        3
folder               2
Name: count, dtype: int64

In [414]:
studentVle.info()
studentVle.sample(7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10655280 entries, 0 to 10655279
Data columns (total 6 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   code_module        object
 1   code_presentation  object
 2   id_student         int64 
 3   id_site            int64 
 4   date               int64 
 5   sum_click          int64 
dtypes: int64(4), object(2)
memory usage: 487.8+ MB


,code_module,code_presentation,id_student,id_site,date,sum_click
6915573,FFF,2013B,532428,526762,135,3
3959918,DDD,2013J,551214,673519,59,6
2083833,CCC,2014B,569108,729822,39,1
1552322,BBB,2014J,2446694,913685,13,3
6839810,FFF,2013B,556783,526737,114,4
2205425,CCC,2014B,2545113,730077,105,1
5394214,EEE,2013J,610873,551010,33,7


In [415]:
find_values_diff(studentVle["id_site"], vleV1["id_site"]) # Есть относительно небольшое кол-во пропусков

array([ 526733,  526818,  527224,  527386,  527399,  527404,  527493,
        527506,  546872,  546897,  547278,  547638,  547639,  547646,
        547648,  547663,  547668,  547682,  547684,  547689,  547692,
        547693,  547718,  547733,  547742,  547771,  547782,  547800,
        547941,  547969,  551014,  551036,  551124,  551159,  551169,
        673527,  673529,  673535,  674130,  674323,  674324,  703977,
        716221,  716675,  716987,  729799,  729803,  729805,  729808,
        729824,  729831,  729835,  729836,  729837,  729838,  729839,
        729840,  729845,  730034,  779622,  790925,  813957,  832764,
        877133,  883074,  909241,  913688,  923834,  923835,  924208,
        924209,  924210,  924211,  924222,  924225,  924383,  924387,
        924388,  936658,  936659,  936763, 1027118, 1032910, 1071060,
       1072098, 1072099, 1072100, 1072236, 1072237, 1072239, 1072240,
       1072252, 1072253, 1073191, 1076553, 1077905])

In [416]:
studentInteractions = pd.merge(
    studentVle,
    vleV1,
    how="inner",
    on=["id_site", "code_module", "code_presentation"]
)
assert(studentInteractions.shape[1] == (studentVle.shape[1] + 1)) # Добавили только activity_type

## Преобразуем данные и создадим новые признаки

In [417]:
studentInteractions.sample(10) # Предлагается поступить глупо и посчитать общее кол-во взаимодействий с модулем

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type
4819905,DDD,2014J,569633,813701,135,1,homepage
7353059,FFF,2013J,467685,716678,49,1,subpage
7509574,FFF,2013J,484574,716238,135,1,homepage
1188203,BBB,2013J,606355,703725,91,1,forumng
6642475,FFF,2013B,556492,526857,45,11,oucontent
3864757,DDD,2013J,600796,674064,20,1,subpage
9119310,FFF,2014J,674691,883092,236,1,subpage
4085881,DDD,2013J,410767,673537,13,3,forumng
3001909,CCC,2014J,1915824,909022,223,2,forumng
4751450,DDD,2014J,485282,813701,-25,15,homepage


In [418]:
# Сделаем группировки
# 1. Общее кол-во кликов в рамках модуля/презентации
clicks_agg = studentInteractions.groupby(["id_student", "code_module", "code_presentation"])["sum_click"].count().reset_index()

# Добавим признаки начала-конца и продоложительность обучения
interaction_start_end = studentInteractions.groupby(["id_student", "code_module", "code_presentation"])["date"].agg(["min", "max"])
interaction_start_end = interaction_start_end.reset_index().rename({"min": "date_start", "max": "date_end"}, axis=1)
interaction_start_end["interaction_interval"] = interaction_start_end["date_end"] - interaction_start_end["date_start"]

interaction_start_end # Возможно в дальнейшем дропнем date_end

,id_student,code_module,code_presentation,date_start,date_end,interaction_interval
0,6516,AAA,2014J,-23,269,292
1,8462,DDD,2013J,-6,118,124
2,8462,DDD,2014J,10,10,0
3,11391,AAA,2013J,-5,253,258
4,23629,BBB,2013B,-6,87,93
...,...,...,...,...,...,...
29223,2698257,AAA,2013J,-9,268,277
29224,2698535,CCC,2014B,-18,149,167
29225,2698535,EEE,2013J,-10,235,245
29226,2698577,BBB,2014J,18,235,217


In [419]:
# Объединим эти группировки в одну
generated_features = pd.merge(
    clicks_agg,
    interaction_start_end,
    on=["id_student", "code_module", "code_presentation"],
    how="inner"
)
assert(generated_features.shape[0] == interaction_start_end.shape[0] == clicks_agg.shape[0])
generated_features.sample(5) # Вроде все ок

,id_student,code_module,code_presentation,sum_click,date_start,date_end,interaction_interval
17316,607473,FFF,2013J,356,-3,67,70
20498,630654,FFF,2014B,244,5,228,223
5262,440319,AAA,2014J,261,-10,159,169
3319,345068,AAA,2013J,213,-4,243,247
21410,634943,FFF,2014J,110,-12,46,58


In [420]:
# Перед объединением посмотрим на основную таблицу в разрезе студента

assesmentsCoursesRegStudentInfo[assesmentsCoursesRegStudentInfo["id_student"] == 2481901]
# Можем обойтись date_submitted и дропнуть date. Так как они по смыслу они одинаковые и есть небольшой геп видимо на проверку

assesmentsCoursesRegStudentInfoV1 = assesmentsCoursesRegStudentInfo.drop("date", axis=1) 
# возможно и date_submitted не нужен так как есть interaction_interval

In [427]:
joined_df = pd.merge(
    assesmentsCoursesRegStudentInfoV1,
    generated_features,
    on=["id_student", "code_module", "code_presentation"],
    how="outer"
)
joined_df.shape # Ок потеряли совсем немного
joined_df.iloc[:, 4:]

,id_assessment,date_submitted,assessment_type,weight,module_presentation_length,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,sum_click,date_start,date_end,interaction_interval
0,1752.0,18.0,TMA,10.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,196.0,-5.0,253.0,258.0
1,1753.0,53.0,TMA,20.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,196.0,-5.0,253.0,258.0
2,1754.0,115.0,TMA,20.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,196.0,-5.0,253.0,258.0
3,1755.0,164.0,TMA,20.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,196.0,-5.0,253.0,258.0
4,1756.0,212.0,TMA,30.0,268.0,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,196.0,-5.0,253.0,258.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180657,37437.0,169.0,TMA,0.0,269.0,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N,181.0,-15.0,219.0,234.0
180658,37438.0,73.0,CMA,0.0,269.0,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N,181.0,-15.0,219.0,234.0
180659,37439.0,150.0,CMA,0.0,269.0,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N,181.0,-15.0,219.0,234.0
180660,37440.0,172.0,CMA,0.0,269.0,F,Yorkshire Region,HE Qualification,50-60%,35-55,0,30,N,181.0,-15.0,219.0,234.0


## Проведем итоговую группировку

In [453]:
joined_df_train = pd.merge(
    train_df,
    joined_df,
    on=["id_student", "code_module", "code_presentation"],
    how="inner"
)

len(find_values_diff(joined_df_train["id_student"], train_df["id_student"])) # Эт хорошо

0             0
1             0
2             0
3             0
4             0
          ...  
144280    26072
144281    26072
144282    26072
144283    26072
144284    26073
Name: ID, Length: 144285, dtype: int64

In [450]:
joined_df_test = pd.merge(
    test_df,
    joined_df,
    on=["id_student", "code_module", "code_presentation"],
    how="inner",
)

len(find_values_diff(joined_df_test["id_student"], test_df["id_student"])) # супер

0

In [454]:
def create_data_set(group):
    # Построим аггрегированную таблицу за основу возьмем признаки которые не меняются между оценками
    aggregated_df = group.sample()[[
        "ID",
        "code_module", "code_presentation", "id_student",
        "date_registration", "module_presentation_length", "gender", "region",
        "highest_education", "imd_band", "age_band", "disability",
        "sum_click", "date_start", "date_end", "interaction_interval"
        ]]
    aggregated_df["sum_score"] = group["weight"].sum().values # Sum баллов за презентацию/модуль
    aggregated_df["maximum_num_of_prev_attempts"] = group["num_of_prev_attempts"].max().values

    # Возьмем середину, но скорее всего все значения одинаковые
    aggregated_df["studied_credits"] = group["studied_credits"].mean().values
    aggregated_df["number_of_assesments"] = group["id_assessment"].count().values # На всякий случай добавим кол-во оценок
    return aggregated_df

In [455]:
train_group = joined_df_train.groupby(["id_student", "code_module", "code_presentation"])
train_dataset = create_data_set(train_group)
assert(len(train_dataset) == len(train_df))

In [456]:
test_group = joined_df_test.groupby(["id_student", "code_module", "code_presentation"])
test_dataset = create_data_set(test_group)
assert(len(test_dataset) == len(test_df))

In [459]:
train_dataset.info() # Ок немного пропусков но не беда
train_dataset.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 26074 entries, 39011 to 8755
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            26074 non-null  int64  
 1   code_module                   26074 non-null  object 
 2   code_presentation             26074 non-null  object 
 3   id_student                    26074 non-null  int64  
 4   date_registration             26037 non-null  float64
 5   module_presentation_length    20640 non-null  float64
 6   gender                        26074 non-null  object 
 7   region                        26074 non-null  object 
 8   highest_education             26074 non-null  object 
 9   imd_band                      25187 non-null  object 
 10  age_band                      26074 non-null  object 
 11  disability                    26074 non-null  object 
 12  sum_click                     23364 non-null  float64
 13  dat

,ID,code_module,code_presentation,id_student,date_registration,module_presentation_length,gender,region,highest_education,imd_band,age_band,disability,sum_click,date_start,date_end,interaction_interval,sum_score,maximum_num_of_prev_attempts,studied_credits,number_of_assesments
88881,15941,DDD,2014J,556514,-45.0,262.0,F,London Region,Lower Than A Level,50-60%,0-35,N,770.0,-25.0,256.0,281.0,200.0,0,60.0,7
102289,18390,CCC,2014B,474283,-22.0,241.0,M,South Region,A Level or Equivalent,80-90%,0-35,N,103.0,-8.0,166.0,174.0,18.0,0,120.0,3
9919,1789,BBB,2013J,572937,-88.0,268.0,F,North Western Region,A Level or Equivalent,60-70%,35-55,N,21.0,-5.0,5.0,10.0,41.0,0,60.0,3
106178,19121,FFF,2014J,686522,-36.0,269.0,M,West Midlands Region,HE Qualification,0-10%,35-55,N,980.0,-16.0,254.0,270.0,100.0,0,90.0,12
2826,512,BBB,2014J,675922,-94.0,262.0,F,East Anglian Region,A Level or Equivalent,80-90%,0-35,N,50.0,-9.0,33.0,42.0,0.0,0,60.0,1
89266,16006,EEE,2014J,656203,-71.0,NaN,M,East Anglian Region,A Level or Equivalent,80-90%,0-35,N,1.0,-7.0,-7.0,0.0,0.0,0,90.0,0
84294,15104,EEE,2014J,687574,-18.0,269.0,M,London Region,A Level or Equivalent,50-60%,0-35,N,568.0,-18.0,257.0,275.0,100.0,0,60.0,4
44955,8052,GGG,2014J,676742,-63.0,NaN,F,London Region,No Formal quals,20-30%,0-35,N,NaN,NaN,NaN,NaN,0.0,0,60.0,0
44941,8049,AAA,2013J,45462,-67.0,268.0,M,Scotland,HE Qualification,30-40%,0-35,N,355.0,-10.0,267.0,277.0,100.0,0,60.0,5
120731,21769,CCC,2014J,676150,-102.0,NaN,M,Scotland,A Level or Equivalent,90-100%,0-35,N,8.0,-10.0,3.0,13.0,0.0,0,60.0,0


In [445]:
test_dataset.info() # Ок немного пропусков, но не критично

<class 'pandas.core.frame.DataFrame'>
Index: 6519 entries, 31103 to 26207
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   code_module                   6519 non-null   object 
 1   code_presentation             6519 non-null   object 
 2   id_student                    6519 non-null   int64  
 3   date_registration             6511 non-null   float64
 4   module_presentation_length    5203 non-null   float64
 5   gender                        6519 non-null   object 
 6   region                        6519 non-null   object 
 7   highest_education             6519 non-null   object 
 8   imd_band                      6295 non-null   object 
 9   age_band                      6519 non-null   object 
 10  disability                    6519 non-null   object 
 11  sum_click                     5864 non-null   float64
 12  date_start                    5864 non-null   float64
 13  dat

### Проведем визуализацию и анализ данных

C этого момента любые изменения в тренировочный датасет(train_dataset) нужно дублировать и в тестовый(test_dataset)